<a href="https://colab.research.google.com/github/syedwaleedhyder/IndoorUserMovementPrediction/blob/master/IndoorUserMovementPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf MovementAAL.zip
!rm -rf MovementAAL/
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00348/MovementAAL.zip
!unzip -o MovementAAL.zip -d MovementAAL

--2019-08-03 14:28:42--  https://archive.ics.uci.edu/ml/machine-learning-databases/00348/MovementAAL.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215028 (210K) [application/x-httpd-php]
Saving to: ‘MovementAAL.zip’

MovementAAL.zip     100%[===================>] 209.99K  --.-KB/s    in 0.1s    

2019-08-03 14:28:42 (1.71 MB/s) - ‘MovementAAL.zip’ saved [215028/215028]

Archive:  MovementAAL.zip
   creating: MovementAAL/dataset/
  inflating: MovementAAL/dataset/MovementAAL_RSS_1.csv  
  inflating: MovementAAL/dataset/MovementAAL_RSS_10.csv  
  inflating: MovementAAL/dataset/MovementAAL_RSS_100.csv  
  inflating: MovementAAL/dataset/MovementAAL_RSS_101.csv  
  inflating: MovementAAL/dataset/MovementAAL_RSS_102.csv  
  inflating: MovementAAL/dataset/MovementAAL_RSS_103.csv  
  inflating: MovementAAL/dataset/MovementAA

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
df1 = pd.read_csv('/content/MovementAAL/dataset/MovementAAL_RSS_1.csv')
df2 = pd.read_csv('/content/MovementAAL/dataset/MovementAAL_RSS_2.csv')

In [4]:
df1.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.90476,-0.48,0.28571,0.30
1,-0.57143,-0.32,0.14286,0.30
2,-0.38095,-0.28,-0.14286,0.35
3,-0.28571,-0.20,-0.47619,0.35
4,-0.14286,-0.20,0.14286,-0.20


In [5]:
print(df1.shape)
print(df2.shape)

(27, 4)
(26, 4)


In [0]:
path = "/content/MovementAAL/dataset/MovementAAL_RSS_"
sequences = list()

for i in range(1,315):
    file_path = path + str(i) + '.csv'
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)

targets = pd.read_csv('/content/MovementAAL/dataset/MovementAAL_target.csv')
targets = targets.values[:,1]

In [7]:
sequences[0]

array([[-0.90476 , -0.48    ,  0.28571 ,  0.3     ],
       [-0.57143 , -0.32    ,  0.14286 ,  0.3     ],
       [-0.38095 , -0.28    , -0.14286 ,  0.35    ],
       [-0.28571 , -0.2     , -0.47619 ,  0.35    ],
       [-0.14286 , -0.2     ,  0.14286 , -0.2     ],
       [-0.14286 , -0.2     ,  0.047619,  0.      ],
       [-0.14286 , -0.16    , -0.38095 ,  0.2     ],
       [-0.14286 , -0.04    , -0.61905 , -0.2     ],
       [-0.095238, -0.08    ,  0.14286 , -0.55    ],
       [-0.047619,  0.04    , -0.095238,  0.05    ],
       [-0.19048 , -0.04    ,  0.095238,  0.4     ],
       [-0.095238, -0.04    , -0.14286 ,  0.35    ],
       [-0.33333 , -0.08    , -0.28571 , -0.2     ],
       [-0.2381  ,  0.04    ,  0.14286 ,  0.35    ],
       [ 0.      ,  0.08    ,  0.14286 ,  0.05    ],
       [-0.095238,  0.04    ,  0.095238,  0.1     ],
       [-0.14286 , -0.2     ,  0.14286 ,  0.5     ],
       [-0.19048 ,  0.04    , -0.42857 ,  0.3     ],
       [-0.14286 , -0.08    , -0.2381  ,  0.15

In [8]:
groups = pd.read_csv('/content/MovementAAL/groups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]
print(groups.shape)

(314,)


In [9]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()

count    314.000000
mean      42.028662
std       16.185303
min       19.000000
25%       26.000000
50%       41.000000
75%       56.000000
max      129.000000
dtype: float64

In [0]:
#Padding the sequence with the values in last row to max length
to_pad = 129
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 60
from keras.preprocessing import sequence
seq_len = 60
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [0]:
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

In [0]:
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

train_target = np.array(train_target)
train_target = (train_target+1)/2

validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2

test_target = np.array(test_target)
test_target = (test_target+1)/2

In [13]:
print(train.shape)
print(validation.shape)
print(test.shape)

(106, 60, 4)
(104, 60, 4)
(104, 60, 4)


In [14]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))

W0803 14:28:47.535533 139729855731584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 14:28:47.580214 139729855731584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 14:28:47.587256 139729855731584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               267264    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 267,521
Trainable params: 267,521
Non-trainable params: 0
_________________________________________________________________


In [18]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=50, batch_size=128, callbacks=[chk], validation_data=(validation,validation_target))

Train on 106 samples, validate on 104 samples
Epoch 1/50
106/106 [==============================] - 1s 9ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 4.2747 - val_acc: 0.5673

Epoch 00001: val_acc improved from -inf to 0.56731, saving model to best_model.pkl
Epoch 2/50
106/106 [==============================] - 0s 1ms/step - loss: 1.3736 - acc: 0.8113 - val_loss: 3.6514 - val_acc: 0.5577

Epoch 00002: val_acc did not improve from 0.56731
Epoch 3/50
106/106 [==============================] - 0s 1ms/step - loss: 0.3871 - acc: 0.9245 - val_loss: 2.9024 - val_acc: 0.5673

Epoch 00003: val_acc did not improve from 0.56731
Epoch 4/50
106/106 [==============================] - 0s 1ms/step - loss: 0.2147 - acc: 0.9623 - val_loss: 2.5226 - val_acc: 0.5577

Epoch 00004: val_acc did not improve from 0.56731
Epoch 5/50
106/106 [==============================] - 0s 1ms/step - loss: 0.0555 - acc: 0.9811 - val_loss: 1.5428 - val_acc: 0.6731

Epoch 00005: val_acc improved from 0.56731 to 0.67308, s

In [19]:
#loading the model and checking accuracy on the test data
model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
accuracy_score(test_target, test_preds)

0.6442307692307693